In [43]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [31]:
# Since our pipeline is not up and running yet, let's just import the traintestsplit function
from python_scripts.traintestsplit import custom_train_test_split

In [32]:
# Read in the cleaned CSV  for football
df_bb = pd.read_csv("cleaned_files/basketball_clean_supervised.csv")
df_fb = pd.read_csv("cleaned_files/football_clean_supervised.csv")

print("Basketball dataset shape: {}, Football dataset shape: {}".format(df_bb.shape,df_fb.shape))


Basketball dataset shape: (137, 19), Football dataset shape: (1263, 19)


In [33]:
# Split both dataset into Train, Test, and Validation sets

X_valid_bb, y_valid_bb, X_train_bb, y_train_bb, X_test_bb, y_test_bb = custom_train_test_split(df_bb)
X_valid_fb, y_valid_fb, X_train_fb, y_train_fb, X_test_fb, y_test_fb = custom_train_test_split(df_fb)

print(X_train_bb.shape, X_test_bb.shape, X_valid_bb.shape)
print(X_train_fb.shape, X_test_fb.shape, X_valid_fb.shape)
#print()

(79, 19) (42, 19) (16, 19)
(728, 19) (379, 19) (156, 19)


In [ ]:
features = ['GRADE','AGE','SKILL','NUMOFF','POS','HEIGHT_IN','WEIGHT_LBS','COLLDIST_MI','NILVAL_LONG_USD','INSTA_LONG','TWIT_LONG','TIK_LONG','TOT_FOL','RECRUIT_YEAR','EXP_MONTHS','EXP_YEARS','ClassificationCode', 'REV_MEN', 'EXP_MEN']

In [34]:
scaler_bb = StandardScaler().fit(X_train_bb[features])
X_train_bb_norm = scaler_bb.transform(X_train_bb[features])
X_test_bb_norm = scaler_bb.transform(X_test_bb[features])

In [37]:
def dummy_reg(X_train,y_train,X_test,y_test):

    mean_dummy = DummyRegressor(strategy="mean").fit(X_train, y_train)
    y_pred = mean_dummy.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    

    return mse, r2

In [45]:
mse, r2 = dummy_reg(X_train_bb_norm,y_train_bb,X_test_bb_norm,y_test_bb)
print("Mean squared error (dummy): {}".format(mse))
print("r2_score (dummy): {}".format(r2))


Mean squared error (dummy): 842225378213.1982
r2_score (dummy): -0.01352419324790377


In [47]:
clf_bb = DecisionTreeRegressor() #maybe insert random state
clf_bb.fit(X_train_bb_norm, y_train_bb)
score_bb = clf_bb.score(X_test_bb_norm, y_test_bb)
print("score: {}".format(score_bb))

score: -0.06411885389022354
